**Writing Smart Contracts 2024: Final Project by Kristina Odermatt (odermk@usi.ch)**

**(6) Voting**:  A club has a membership list with the public keys of all members. Write a smart contract that allows members to vote for one of three candidates (Rossi, Smith or Meier). Extension: (1) Ensure that voting is only possible in a certain time frame.




# **Notebook 1: Creating and deploying the smart contract**

## Setup 

In [1]:
# loading shared code and credentials
from algo_util import *

cred = load_credentials("./credentials_project")
members = cred['Members']
club = cred['Club']


In [2]:
# importing necessary modules and functions from the Algorand SDK
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [3]:
# importing all the contents from the pyteal module
from pyteal import *

In [4]:
# initialize the algod client (Testnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

38927104

In [5]:
# print member's public key
for member_id, member_details in cred['Members'].items():
    public_key = member_details['public']
    print(f"{member_id}: {public_key}")


Member_1: 4PTDQQ2P5L7GR4TQQOUPEKMZIYIR3GUN27TMG2J4WRTCL64RCOEZVB3YDI
Member_2: AH4EIE2RRD47FAMDCTQXUC3SP6HK5XQTIFNOGANNPRG6Q6UEKUOWOXBFUU
Member_3: PSETUPGFLYJIW6RLSMVWRNIM4E4FDAU43HMEHVGJEGPHM6YMWF5UOMUEVU
Member_4: DRKOGSHJTEMVXLSZ3BRZTY4KLFQ2XM2ASB4ZRDIQE2PE3OW54PG3VI7X64
Member_5: VPIXH54SZ3YC6AEIGUABMXOTAAMVNXLE6IINBOLPB4MQRHRP36ACEK33UQ
Member_6: YTCBPJA5VVI32KFOAUN7KXRPQBDA53SVSEEWDFC2VSP7ZV5QODPCGNWFTE
Member_7: PVNIKBBQKUICXIAY64GLW5X3JJLB3URYQFZJPQ5LLGKF4KXC4MGYW6E344
Member_8: L437EAAM32GCXHF7NHDJT57ETQLGLLGZ74ORU5U2R4YM4OXUI6ZARW5YFY
Member_9: MDBVAFK5KK2UVDZKQNREXUNXUERGHOMISZMVV3P5EUDHD4I4ZDBGRL3XFQ
Member_10: KPOVOUJYWZEADLWI2CNOWKBHVIG3BPBB2H2HAK4F4NUIXFV5WKT4EEFKLI
Member_11: OSS6YJ7OT7W55JGC2ZIEPYV6HKKVI3KT6C326T4NOSAFRCOW7Y3PAXBJOA
Member_12: QQO2SO3SO6U3QBCPB3QBP44625G42CSY7YNIPMVZRHWZPXMGJDJDMX5NPU
Member_13: 4FK2DU2FX2BEILN22TDEM7OABP777SBITGK25CQUUX3EUGY5CVI53UWJRM
Member_14: RFHODOL22YZ733SQV5OOLASUMQRS22BUPF23KIRDLEP37OWR47EAW5UH4E
Member_15: 7NE2HI2AYRPNNT437Q

In [6]:
# print club's public key
(club['public'])

'KPH4VDOVVVP3BWOUJYIETXGDMS3ZQR3DHCQ7HBEHTVCKIN264VW7WUJZ5E'

## Fund accounts
* ‼️ Prior to using the smart contract it is essential to fund Members accounts and Club account for transaction fees and smart contract interactions

* Visit https://dispenser.testnet.aws.algodev.network and select **Currency 5 Algo**


## 1.1 Creating the Smart Contract

### Step 1: Define the Approval Program

In [7]:
# find current last round from the Algorand blockchain
last_round = algod_client.status()['last-round']

# set the voting start round to be the next round after the current last round
voting_start_round = Int(last_round + 1)  # Use Int for PyTeal compatibility

# set the voting end round to be 200 rounds after the voting start round
voting_end_round = voting_start_round + Int(200)  # Adding 200 rounds as the voting period, which shoud be approx. 15 minutes

In [8]:
# define the public key of the club account to distinguish it from members accounts
club_account_pk = Addr(club['public'])

In [9]:
# handle the creation of the smart contract instance
handle_creation = Seq([
    # initialize global state variables
    App.globalPut(Bytes("Note"), Bytes("Welcome to the Voting System")),  # welcome note
    App.globalPut(Bytes("Members"), Int(15)),  # total number of club members is fixed to 15 before and after voting
    App.globalPut(Bytes("TotalVotes"), Int(0)),  # counter for total votes cast
    App.globalPut(Bytes("StartRound"), voting_start_round),  # start of the voting period
    App.globalPut(Bytes("EndRound"), voting_end_round),  # end of the voting period
    Return(Int(1))  # successfully created the app
])

# handle user opt-in to the smart contract (enabling per-user local state)
handle_optin = Seq([
    # initialize 'has_voted' local state for the user as 0 (false)
    App.localPut(
        Int(0), 
        Bytes("has_voted"), 
        Int(0)
    ),
    Return(Int(1))  # opt-in succeeded
])

# handle account closeout from the smart contract
handle_closeout = Return(Int(1))  # No special action required on closeout

# handle request to update the smart contract code
handle_updateapp = Return(Int(0))  # disallow updates to the app by default

# handle request to delete the smart contract
handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()  # allow deletion only by the contract creator which is the club
)

# handle no-op transactions (main logic for voting)
handle_noop = Seq([
    # conditional branch for handling voting logic
    If(
        And(
            # check if exactly one argument is passed (candidate name)
            Txn.application_args.length() == Int(1),
            # ensure current round is within the voting period
            Global.round() >= App.globalGet(Bytes("StartRound")),
            Global.round() <= App.globalGet(Bytes("EndRound")),
            # ensure the sender is not the club account (prevent tampering)
            Txn.sender() != club_account_pk,
            # ensure the sender has not already voted
            App.localGet(Int(0), Bytes("has_voted")) == Int(0)
        ),
        Seq([
            # validate candidate name and process vote
            If(
                Or(
                    Txn.application_args[0] == Bytes("Rossi"),
                    Txn.application_args[0] == Bytes("Smith"),
                    Txn.application_args[0] == Bytes("Meier")
                ),
                Seq([
                    # increment vote count for the selected candidate
                    App.globalPut(Txn.application_args[0], App.globalGet(Txn.application_args[0]) + Int(1)),
                    
                    # increment total number of votes cast
                    App.globalPut(Bytes("TotalVotes"), App.globalGet(Bytes("TotalVotes")) + Int(1)),
                    
                    # mark the voter as having voted
                    App.localPut(Int(0), Bytes("has_voted"), Int(1)),
                    Return(Int(1))  # Vote successfully recorded
                ]),
                Return(Int(0))  # return failure for invalid candidate name
            )
        ]),
        Return(Int(0))  # return failure if conditions for voting are not met
    )         
])

In [10]:
# this function routes the execution to different blocks of code based on the type of transaction being processed
member_vote_approval_pyteal = Cond(
    
    # if the application is being created, execute the handle_creation logic
    [Txn.application_id() == Int(0),                       handle_creation],
    
    # if a user opts in to the application, execute the handle_optin logic
    [Txn.on_completion()  == OnComplete.OptIn,             handle_optin],
    
    # if a user is closing out of the application, execute the handle_closeout logic
    [Txn.on_completion()  == OnComplete.CloseOut,          handle_closeout],
    
    # if there's an attempt to update the application, execute the handle_updateapp logic
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    
    # if there's a request to delete the application, execute the handle_deleteapp logic
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    
    # for any other no-op transactions (like voting), execute the handle_noop logic
    [Txn.on_completion()  == OnComplete.NoOp,              handle_noop],
)


#### Compile PyTEAL -> TEAL

In [11]:
# compile the PyTeal into TEAL code
member_vote_approval_teal = compileTeal(member_vote_approval_pyteal, mode=Mode.Application, version=5)
print(member_vote_approval_teal)


#pragma version 5
txn ApplicationID
int 0
==
bnz main_l16
txn OnCompletion
int OptIn
==
bnz main_l15
txn OnCompletion
int CloseOut
==
bnz main_l14
txn OnCompletion
int UpdateApplication
==
bnz main_l13
txn OnCompletion
int DeleteApplication
==
bnz main_l12
txn OnCompletion
int NoOp
==
bnz main_l7
err
main_l7:
txn NumAppArgs
int 1
==
global Round
byte "StartRound"
app_global_get
>=
&&
global Round
byte "EndRound"
app_global_get
<=
&&
txn Sender
addr KPH4VDOVVVP3BWOUJYIETXGDMS3ZQR3DHCQ7HBEHTVCKIN264VW7WUJZ5E
!=
&&
int 0
byte "has_voted"
app_local_get
int 0
==
&&
bnz main_l9
int 0
return
main_l9:
txna ApplicationArgs 0
byte "Rossi"
==
txna ApplicationArgs 0
byte "Smith"
==
||
txna ApplicationArgs 0
byte "Meier"
==
||
bnz main_l11
int 0
return
main_l11:
txna ApplicationArgs 0
txna ApplicationArgs 0
app_global_get
int 1
+
app_global_put
byte "TotalVotes"
byte "TotalVotes"
app_global_get
int 1
+
app_global_put
int 0
byte "has_voted"
int 1
app_local_put
int 1
return
main_l12:
txn Sender
globa

### Step 2: Define Clear State Program

In [12]:
# define the Clear State Program using PyTeal
clear_state_program_pyteal = Return(Int(1))  # Simple clear state logic

# compile the PyTeal Clear State Program into TEAL code
member_vote_clear_teal = compileTeal(clear_state_program_pyteal, mode=Mode.Application, version=3)

print(member_vote_clear_teal)


#pragma version 3
int 1
return


#### Compile TEAL -> Bytecode¶

In [13]:
# compile the Approval Program TEAL code into bytecode
approval_program_response = algod_client.compile(member_vote_approval_teal)
approval_program_compiled = base64.b64decode(approval_program_response['result'])
print("Approval Program Compiled to Bytecode")

# compile the Clear State Program TEAL code into bytecode
clear_program_response = algod_client.compile(member_vote_clear_teal)
clear_program_compiled = base64.b64decode(clear_program_response['result'])
print("Clear State Program Compiled to Bytecode")


Approval Program Compiled to Bytecode
Clear State Program Compiled to Bytecode


## 1.2 Deploying the Smart Contract

In [14]:
# retrieve the current suggested transaction parameters from the Algorand network
sp = algod_client.suggested_params()


In [15]:
# define state schemas for the smart contract's global and local state
global_ints = 7    # for "Rossi", "Smith", "Meier" votes and "TotalVotes"
global_bytes = 1   # for "Note"
voting_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 1    # for "has_voted" only
local_bytes = 0    # no local byte slices are required
voting_local_schema = transaction.StateSchema(local_ints, local_bytes)


In [16]:
# step 1: create an application transaction to deploy the smart contract on the Algorand blockchain
txn = transaction.ApplicationCreateTxn(
      sender = club['public'],
      sp = sp,
      on_complete = transaction.OnComplete.NoOpOC,
      approval_program = approval_program_compiled,
      clear_program = clear_program_compiled,
      global_schema = voting_global_schema,
      local_schema = voting_local_schema
    )


In [17]:
# step 2: sign transaction
stxn = txn.sign(club['private'])

# step 3: send
txid=algod_client.send_transactions([stxn])

# step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  38927107.
Waiting for round 38927107 to finish.
Transaction JQIQZHAU4GFTJXNCPR4DJC5YHZQZGJLDNPLOYXANTEHO7H2WCBBA confirmed in round 38927108.


In [18]:
# create new app_id which is unique for each smart contract
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

Created new app-id: 640731779


## 1.3 Checking state of Smart Contract

In [19]:
# inspect global state
read_global_state(algod_client,app_id) 

{'EndRound': 38927305,
 'Members': 15,
 'Note': 'Welcome to the Voting System',
 'StartRound': 38927105,
 'TotalVotes': 0}

In [20]:
# insepct local state for Members
member_name = 'Member_1'                    # <-- ‼️ UPDATE/CHANGE Member's name for each 

read_local_state(algod_client,members[member_name]['public'],app_id)

{}

In [21]:
# insepct local state Club
read_local_state(algod_client,club['public'],app_id)

{}

In [22]:
# program code immediately visible on the web
print(f"{cred['explore_main']}application/{app_id}")

https://testnet.explorer.perawallet.app/application/640731779
